In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2 as pg

In [3]:
conn = pg.connect(
    host='localhost',
    database='phoenix',
    port=5432,
    user='postgres',
    password='adminphoenix'
)
cursor = conn.cursor()

In [4]:
churn = pd.read_sql_query("""SELECT * FROM churnbase""",
                          conn)
cursor.close()
conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_7356\4111480195.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  churn = pd.read_sql_query("""SELECT * FROM churnbase""",
